In this project, web scraping was aimed through Selenium. The data of a list of randomly chosen 581 places were extracted from Restaurantguru.com, a website that have the customer rate data from Google, Tripadvisor, Zomato, Foursquare, Yelp and Facebook. Later these rates and number of rates will be used to segment these places in another project.

In [1]:
# --- Kütüphaneler ---
import pandas as pd
from selenium import webdriver
import time

pd.set_option('display.max_columns', None)
data= pd.read_excel(r"C:\Users\PC\Documents\places.xlsx")

In [2]:
# --- Düzenleme ---
def data_prep(dataset):
    dataset["name_rg"] = ""
    dataset["price_check_rg"] = ""
    dataset["rank_rg"] = ""
    dataset["rating_rg"] = ""
    dataset["price_rg"] = ""
    dataset["address_rg"] = ""
    dataset["features_rg"] = ""
    dataset["website_rg"] = ""
    dataset["instagram_rg"] = ""
    dataset["cuisine_rg"] = ""
    dataset["yelp_rg"] = ""
    dataset["trip_rg"] = ""
    

def data_check_col(dataset):
    dataset["checkpoint"] = False
    
def add_info(path,search="Regular",path_type="css",get_attribute="none"): 
    try:
        if search=="website":
            info = rg_browser.find_element_by_css_selector(path).get_attribute("href")
            return info
        elif path_type=="xpath" and get_attribute=="title":
            info = rg_browser.find_element_by_xpath(path).get_attribute(get_attribute)
            return info
        elif path_type=="xpath":
            info = rg_browser.find_element_by_xpath(path)
            return info
        elif get_attribute=="class":
            info = rg_browser.find_element_by_css_selector(path).get_attribute(get_attribute)
            return info
        else:
            info = rg_browser.find_element_by_css_selector(path)
            return info.text
    except:
        return "No-Info"

data_prep(data)    
data_check_col(data)

In [3]:
data.head()

,place_name,il,ilçe,Adres,name_rg,price_check_rg,rank_rg,rating_rg,price_rg,address_rg,features_rg,website_rg,instagram_rg,cuisine_rg,yelp_rg,trip_rg,checkpoint
0,11 Kadıköy,İstanbul,Kadıköy,CAFERAĞA MAH ARAYICI BAŞI SOK NO 11 A-1 KADIKO...,,,,,,,,,,,,,False
1,3g pub,İzmir,Konak,CUMHURİYET BULVARI NO:18 KONAK IZMIR,,,,,,,,,,,,,False
2,45 Bar Kadikoy,İstanbul,Kadıköy,İSTANBUL KADIKÖY KADIKOY ISTANBUL,,,,,,,,,,,,,False
3,6:45 Losers Club,İzmir,Konak,1482 SOKAK NO:10 KONAK IZMIR,,,,,,,,,,,,,False
4,Adabeyi Balık Restoran,İzmir,Konak,ATATÜRK Cd.AKDENİZ Mh. No:19/d,,,,,,,,,,,,,False


In [4]:
from selenium import webdriver
import time
from selenium.webdriver.firefox.options import Options as FirefoxOptions
options = FirefoxOptions()
options.add_argument("--headless")
count = 0

for i in range(len(data)):
    if data.loc[i,"checkpoint"] != True:
            rg_browser = webdriver.Firefox(executable_path =r"C:\Users\PC\Downloads\geckodriver-v0.31.0-win64\geckodriver.exe")

            if data.loc[i,"ilçe"] == "Konak":
                rg_browser.execute_script(f"location.href='https://restaurantguru.com/restaurant-Izmir-t1';")
            elif data.loc[i,"ilçe"] == "Kadıköy":
                rg_browser.execute_script(f"location.href='https://restaurantguru.com/search/Kad%C4%B1k%C3%B6y-lat40.99047-lon29.02169?set=5&distance=5000&skip_geo=1';")
            elif data.loc[i,"ilçe"] == "Şişli":
                rg_browser.execute_script(f"location.href='https://restaurantguru.com/search/Ni%C5%9Fanta%C5%9F%C4%B1-lat41.05169-lon28.9917?distance=10000';")
            else:
                rg_browser.execute_script(f"location.href='https://restaurantguru.com/Buyukcekmece#restaurant-list';")
                            
            rg_browser.switch_to.window(rg_browser.window_handles[0])
            time.sleep(6)
            try: 
                info = rg_browser.find_element_by_css_selector(".form_header")
                if info.text == "Our systems have detected unusual traffic from your computer network.":
                    print("!! ROBOT TESTI !!")
                    break
            except:
                
                search_bar = rg_browser.find_element_by_css_selector("#q")  
                search_bar.click()
                search_bar.send_keys(data.loc[i,"place_name"]) 
                time.sleep(3)
                search_button = rg_browser.find_element_by_xpath("/html/body/div[1]/div[1]/div/form/button")
                search_button.click()
                time.sleep(3)
                
                data.loc[i,"name_rg"] = add_info('/html/body/div[3]/div[1]/div[3]/div[4]/div/div[1]/div[1]/div/div[2]/div[1]/div/a',path_type="xpath",get_attribute="title")
                data.loc[i,"price_check_rg"] = add_info("/html/body/div[3]/div[1]/div[3]/div[4]/div/div[1]/div[1]/div/div[2]/div[2]/div[4]/span", path_type="xpath",get_attribute="title")
                try:
                    
                    rg_browser.find_element_by_xpath('/html/body/div[3]/div[1]/div[3]/div[4]/div/div[1]/div[1]/div/div[2]/div[1]/div/a').click()

                    anasayfa = rg_browser.window_handles[0]
                    acilan_sayfa = rg_browser.window_handles[1]
                    rg_browser.switch_to.window(acilan_sayfa)

                    time.sleep(4)
                    
                    try:
                        info = rg_browser.find_element_by_css_selector(".form_header")
                        if info.text == "Our systems have detected unusual traffic from your computer network.":
                            print("!! ROBOT TESTI !!")
                            break
                    except:
                        data.loc[i, "rank_rg"] = add_info(".r_prime > div:nth-child(1) > a:nth-child(1)")
                        data.loc[i, "rating_rg"] = add_info("#ratings")
                        data.loc[i, "price_rg"] = add_info("div.short_info:nth-child(2)")
                        data.loc[i, "address_rg"] = add_info("#info_location")
                        data.loc[i, "features_rg"] = add_info(".features_block")
                        data.loc[i, "website_rg"] = add_info(".website")
                        data.loc[i, "instagram_rg"] = add_info(".instagram")
                        data.loc[i, "cuisine_rg"] = add_info("#content > div.columns_wrapper.relative.clear > div.right_column_wrapper > div.right_top > div.wrapper_main_info > div.cuisine_wrapper")
                        data.loc[i, "yelp_rg"] = add_info("#ratings > div.wrapper_ratings.notranslate > a.row.yelp.rating_list_right > div.right > div", get_attribute="class")
                        data.loc[i, "trip_rg"] = add_info("#ratings > div.wrapper_ratings.notranslate > div > div.right > div > div", get_attribute="class")
                        data.loc[i,"checkpoint"] = True
                        rg_browser.close()
                        rg_browser.switch_to.window(anasayfa)
                        
                except:
                    data.loc[i,"name_rg"] = "No-Info"
                    data.loc[i,"price_check_rg"] = "No-Info"
                    data.loc[i,"rank_rg"] = "No-Info"
                    data.loc[i,"rating_rg"] = "No-Info"
                    data.loc[i,"price_rg"] = "No-Info"
                    data.loc[i,"address_rg"] = "No-Info"
                    data.loc[i,"features_rg"] = "No-Info"
                    data.loc[i,"website_rg"] = "No-Info"
                    data.loc[i,"instagram_rg"] = "No-Info"
                    data.loc[i,"cuisine_rg"] = "No-Info"
                    data.loc[i,"yelp_rg"] = "No-Info"
                    data.loc[i,"trip_rg"] = "No-Info"
                    data.loc[i,"checkpoint"] = True
                    
                
                rg_browser.close()

    else:
        print(f"\rCount: {count}", end="")
        count+=1
        continue

C:\Users\PC\AppData\Local\Temp/ipykernel_11864/729264610.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  rg_browser = webdriver.Firefox(executable_path =r"C:\Users\PC\Downloads\geckodriver-v0.31.0-win64\geckodriver.exe")
C:\Users\PC\AppData\Local\Temp/ipykernel_11864/729264610.py:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  info = rg_browser.find_element_by_css_selector(".form_header")
C:\Users\PC\AppData\Local\Temp/ipykernel_11864/729264610.py:30: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search_bar = rg_browser.find_element_by_css_selector("#q")  #.what_with_highlight
C:\Users\PC\AppData\Local\Temp/ipykernel_11864/729264610.py:34: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
 

In [5]:
data.head()

,place_name,il,ilçe,Adres,name_rg,price_check_rg,rank_rg,rating_rg,price_rg,address_rg,features_rg,website_rg,instagram_rg,cuisine_rg,yelp_rg,trip_rg,checkpoint
0,11 Kadıköy,İstanbul,Kadıköy,CAFERAĞA MAH ARAYICI BAŞI SOK NO 11 A-1 KADIKO...,11 Kadıköy,No-Info,#221 of 72430 restaurants in Istanbul,Ratings of 11 Kadıköy\nZomato\n4.6 / 5\n1097\n...,$$$$ Price range per person TRY 170 - TRY 410,"Address\nKadıköy\nArayıcıbaşı Sk. No:11, Istan...",Features\nСredit cards accepted\nOutdoor seati...,No-Info,@11kadikoy,Cocktail bars,No-Info,No-Info,True
1,3g pub,İzmir,Konak,CUMHURİYET BULVARI NO:18 KONAK IZMIR,3G Cafe,Expensive (average price TRY 410 - TRY 830),#1732 of 12479 cafes in Izmir,Ratings of 3G Cafe\nYelp\nNot rated yet\nFours...,$$$$ Price range per person TRY 410 - TRY 830,Address\nÇankaya\nKonak Mahallesi İzmirlioğlu ...,Features\nСredit cards accepted\nOutdoor seati...,No-Info,No-Info,No-Info,no_rating,No-Info,True
2,45 Bar Kadikoy,İstanbul,Kadıköy,İSTANBUL KADIKÖY KADIKOY ISTANBUL,45 Bar Kadikoy,No-Info,#1309 of 11620 pubs & bars in Istanbul,Ratings of 45 Bar Kadikoy\nZomato\n2.4 / 5\n83...,$$$$ Price range per person TRY 170 - TRY 410,"Address\nKadıköy\nMiralay Nazım Sk. No:19/A, I...",Features\nСredit cards accepted\nWi-Fi\nBookin...,Website\n45likkadikoy.com,@45likcomtr,No-Info,No-Info,No-Info,True
3,6:45 Losers Club,İzmir,Konak,1482 SOKAK NO:10 KONAK IZMIR,6:45 Losers Club,Moderate (average price TRY 170 - TRY 410),#47 of 599 clubs in Izmir,Ratings of 6:45 Losers Club\nGoogle\n955,$$$$ Price range per person TRY 170 - TRY 410,"Address\nÇankaya\nMuzaffer İzgü Sk. No:10, Izm...",Features\nNo delivery\nOutdoor seating\nNo tak...,No-Info,No-Info,No-Info,No-Info,No-Info,True
4,Adabeyi Balık Restoran,İzmir,Konak,ATATÜRK Cd.AKDENİZ Mh. No:19/d,Adabeyi Balık Restoran,Expensive (average price TRY 410 - TRY 830),#320 of 18291 restaurants in Izmir,Ratings of Adabeyi Balık Restoran\nYelp\n5\nZo...,$$$$ Price range per person TRY 410 - TRY 830,Address\nÇankaya\nKonak Atatürk Cad. Konak Pie...,Features\nСredit cards accepted\nOutdoor seati...,No-Info,No-Info,"Seafood, Turkish",yelp_block stars5,fill rate4_5,True


In [6]:
data.to_csv("places_final.csv")